Black - Scholes European Option Pricer -- With analytical formula and Monte Carlo simulation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from scipy.stats import norm

In [3]:
from scipy.optimize import brentq   # root finidng algorithm

In [5]:
def bs_prices(S, K, r, q, sigma, T, option_type='call'):
    if T<= 0 or sigma <= 0:   # if time to maturity is zero, the option value must just equal its intrinsic value:
        if option_type== 'call':
            return max(0,0, S*np.exp(-q*T) - K*np.exp(-r*T))   # max (S e^{-qT} -K e^{-r T}, 0)
        else:
            return max(0.0, K*np.exp(-r*T) - S*np.exp(-q*T))  # max( 0 , K e^{-r T} - S e^{-q T} )
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))  # q is dividend yield: (S/K + (r - q + sigma^2/2)T)/(sigma sqrt[T])
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1)
    
    

In [6]:
def mc_prices(S, K, r, q, sigma, T,option_type='call', n_paths=10000,antithetic = False, moment_matching=False, seed=123):
    rng = np.random.default_rng(seed)
    n= n_paths
    if antithetic:
        m = (n+1)//2
        Z = rng.standard_normal(m)
        Z = np.concatenate([Z, -Z[:n-m]])
    else:
        Z= rng.standard_normal(n)

    if moment_matching:
        meanZ = Z.mean()
        stdZ = Z.std(ddof =0)
        if stdZ == 0:
            raise ValueError("degenerate normal draws")
        Z = (Z - meanZ)/stdZ
    
    drift = (r - q - 0.5*sigma**2)*T
    diffusion = sigma*np.sqrt(T)*Z
    S_T = S*np.exp(drift + diffusion)

    if option_type=='call':
        payoffs = np.maximum(S_T - K, 0.0)
    else:
        payoffs = np.maximum( K - S_T, 0.0)
    discounted = np.exp(-r*T)*payoffs
    price = discounted.mean()
    stderr = discounted.std(ddof = 1)/np.sqrt(n)
    return price, stderr

# discounted.std(ddof=1) → this is σ/Sqrt(n), the sample standard deviation.
#ddof=1 means divide by (n-1) instead of n, i.e. the unbiased estimator of variance.
# Divide by Sqrt[n]-> gives SE of sample. 



In [13]:
def implied_vol(market_price, S,K,r,q,T,option_type='call', sigma_bounds=(1e-6,5.0), tol=1e-8):
    def f(sigma):
        return bs_prices(S, K, r, q, T, sigma, option_type)- market_price
    low, high = sigma_bounds
    if f(low)* f(high) > 0:
        raise ValueError("Implied Volatilitiy not braketed. Try larger bounds or check ip price")
    vol = brentq(f,low,high,xtol=tol,rtol=tol, maxiter=1000)
    return vol
# Implied volatility = the volatility input to the Black–Scholes formula that reproduces the observed market option price.
# compute the diff and if f(sigma) = 0, then sigma is the implied volatility.
# Start with a lower bound (1e-6) and upper bound (5.0) for σ. This brackets the root: we assume the implied vol lies somewhere in this range.
# Root-finding methods like Brent’s method require the function to have opposite signs at the endpoints (f(low) and f(high)), 
#otherwise there’s no guarantee a solution exists inside.

In [14]:
if __name__ == '__main__':
    S= 100.0
    K = 100.0
    r = 0.05
    q = 0.0
    sigma = 0.2
    T = 0.5

    call_analytical = bs_prices(S, K,r, q, sigma, T, 'call')
    put_analytical = bs_prices(S, K, r,q,sigma, T, 'put')
    print("Analytical call:" , call_analytical)
    print("Analytical put:", put_analytical)

    #mc_prices(S, K, r, q, sigma, option_type='call', n_paths=10000,antithetic = False, moment_matching=False, seed=123):
    mc_plain = mc_prices(S,K, r,q,sigma,T,'call', n_paths=200000, antithetic= False, moment_matching= False)
    print("MC plain call price, stderr:", mc_plain)

    mc_anti = mc_prices(S,K, r,q,sigma, T,'call', n_paths=200000, antithetic= True, moment_matching= False)
    print("MC anti call price, stderr:", mc_anti)

    mc_both = mc_prices(S,K, r,q,sigma,T,'call', n_paths=200000, antithetic= True, moment_matching= True)
    print("MC both call price, stderr:", mc_both)

    market_price = call_analytical
    implied = implied_vol(market_price, S, K, r, q, T,'call')
    print("implied vol (should equal sigma):", implied)

    

Analytical call: 6.888728577680624
Analytical put: 4.41971978051388
MC plain call price, stderr: (6.8721379862800935, 0.0218833226061422)
MC anti call price, stderr: (6.88279878627146, 0.02188937923968412)
MC both call price, stderr: (6.884056049434203, 0.02189383580083499)
implied vol (should equal sigma): 0.11064737197975866
